In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import sys
import pandas_redshift as pr
import json
import requests

In [2]:
job_start=dt.datetime.utcnow()

#### Intake Data

In [3]:
input_df=pd.read_excel("/Users/peter.gurka/Google Drive/My Drive/Self Serve Reporting/API 1 User Demographic Data Append/Intake Sheet - API1.xlsx", sheet_name='Data', usecols='A:C')
request_df=pd.read_excel("/Users/peter.gurka/Google Drive/My Drive/Self Serve Reporting/API 1 User Demographic Data Append/Intake Sheet - API1.xlsx", sheet_name='Request')

In [4]:
request_df

,Field,Value
0,Initials of Requester,HZ
1,Subject (ClientTopic),TestingforPeter3
2,SALT,tF$SOQjjIoFi


In [5]:
input_df.head()

,FieldNames,UserID,Click_UUID
0,postal_code,16532237,NaN
1,date_of_birth,21116858,NaN
2,ethnicity,21789953,NaN
3,gender,21284787,NaN
4,firstname,21346632,NaN


In [6]:
date = dt.datetime.today().date()

initials = request_df['Value'][0]
subject = request_df['Value'][1]
salt = request_df['Value'][2]

In [7]:
from random import randint
rand=randint(1, 99)

In [8]:
input_df['FieldNames']=input_df['FieldNames'].replace('date_of_birth', 
                                                      'datediff(year, date_of_birth, current_date) as age')
for children in ['child_1_dob','child_2_dob','child_3_dob','child_4_dob','child_5_dob','child_6_dob',
                 'child_7_dob','child_8_dob','child_9_dob']:
    input_df['FieldNames']=input_df['FieldNames'].replace(children, 
                           'datediff(year, coalesce(date({0}), current_date), current_date) as {0}_age'.format(children))
fieldnames_list = input_df['FieldNames'].dropna().to_list()
fieldnames = ', '.join(fieldnames_list).strip('"')

#### Post on Slack

In [9]:
webhook_url = 'https://xxx'

In [10]:
slack_message_start = """{0}'s API1 request for {1} has started""".format(initials, subject)
slack_data = {'text': slack_message_start}

In [11]:
response = requests.post(
    webhook_url, data = json.dumps(slack_data),
    headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    raise ValueError(
        'Request to slack returned an error %s, the response is:\n%s'
        % (response.status_code, response.text))

#### Insert User ID List 

In [1]:
server='xxx'
port='xxx'
dbase='xxx'
user='xxx'
password='xxx'

pr.connect_to_redshift(dbname = dbase,
                       host = server,
                       port = port,
                       user = user,
                       password = password)

NameError: name 'pr' is not defined

In [13]:
temptable="""API1_UserID_{0}_{1}_{2}_{3}""".format(date.strftime("%Y-%m-%d").replace("-",""), initials, subject, rand)
temptable

'API1_UserID_20220214_HZ_TestingforPeter3_93'

#### User ID/Click UUID 

In [14]:
if len(input_df['UserID'].dropna())>0:    
    id_type = 'user_id'
elif len(input_df['Click_UUID'].dropna())>0:
    id_type = 'click_uuid'

In [15]:
if id_type == 'user_id':
    input_df['UserID_Insert'] = "('" + input_df['UserID'].dropna().astype(str) + "')"
elif id_type == 'click_uuid':
    input_df['UserID_Insert'] = "('" + input_df['Click_UUID'].dropna().astype(str) + "')"
    fieldnames = 'a.click_uuid,' + fieldnames

userids_list = input_df['UserID_Insert'].dropna().drop_duplicates().to_list()
userids = ', '.join(userids_list).strip('"')

In [16]:
if id_type == 'user_id':
    click_uuid_str = """
    CREATE TEMP TABLE {temptable} AS (
    SELECT user_id::INTEGER
    FROM {temptable}_int);
    """.format(temptable=temptable)

elif id_type == 'click_uuid':
    click_uuid_str = """
    CREATE TEMP TABLE {temptable} AS (
    SELECT t1.user_id::VARCHAR as click_uuid,
           t2.user_id::INTEGER as user_id
    FROM {temptable}_int t1
    LEFT JOIN prd_edw.vw_uab_silver t2
    ON t1.user_id = t2.click_uuid);
    """.format(temptable=temptable)

In [17]:
temptable_insert="""
DROP TABLE IF EXISTS {temptable}_int;

CREATE TEMP TABLE {temptable}_int (user_id varchar(256));

INSERT INTO {temptable}_int VALUES {userids};

{click_uuid_str}

SELECT COUNT(*) FROM {temptable};
""".format(temptable=temptable, userids=userids, click_uuid_str=click_uuid_str)

In [18]:
print(temptable_insert)


DROP TABLE IF EXISTS API1_UserID_20220214_HZ_TestingforPeter3_93_int;

CREATE TEMP TABLE API1_UserID_20220214_HZ_TestingforPeter3_93_int (user_id varchar(256));

INSERT INTO API1_UserID_20220214_HZ_TestingforPeter3_93_int VALUES ('16532237'), ('21116858'), ('21789953'), ('21284787'), ('21346632'), ('21786339'), ('21585290'), ('22000649'), ('21733609'), ('21986164');


    CREATE TEMP TABLE API1_UserID_20220214_HZ_TestingforPeter3_93 AS (
    SELECT user_id::INTEGER
    FROM API1_UserID_20220214_HZ_TestingforPeter3_93_int);
    

SELECT COUNT(*) FROM API1_UserID_20220214_HZ_TestingforPeter3_93;



In [19]:
start=dt.datetime.now()

insert_check=pr.redshift_to_pandas(temptable_insert)
print("number of rows inserted:", insert_check['count'][0])
#cursor.execute(temptable_insert) 

end=dt.datetime.now()
print('minutes to complete insert:', (end-start).seconds//60)

number of rows inserted: 10
minutes to complete insert: 0


In [20]:
#input_df[['UserID']].to_sql(temptable, engine, index=False, if_exists='append')

In [21]:
check="""
SELECT * FROM {temptable} LIMIT 5;
""".format(temptable=temptable)
pr.redshift_to_pandas(check)

,user_id
0,16532237
1,21116858
2,21789953
3,21284787
4,21346632


#### Data Request Pull

In [22]:
tablename = """stg_data_genie.API1_{0}_{1}_{2}_{3}""".format(date.strftime("%Y-%m-%d").replace("-",""), initials, subject, rand)
tablename

'stg_data_genie.API1_20220214_HZ_TestingforPeter3_93'

In [23]:
data_requests_log_id="""
SELECT MAX(id)+1 as id_val from stg_data_genie.data_requests_log;"""
id_val=pr.redshift_to_pandas(data_requests_log_id)
id_val=id_val['id_val'][0]
id_val

134

In [24]:
data_request1="""
drop table if exists {tablename};

create table {tablename} as (
select  md5(concat(a.user_id, '{salt}')) AS hashed_user_id,
        a.user_id as user_id,
        {fieldnames}
from {temptable} a
join xxx b
  on a.user_id = b.user_id
join xxx c
  on a.user_id = c.user_id
left join xxx d
  on a.user_id=d.user_id
  and d.user_id IS NULL
LEFT JOIN xxx s
    ON a.user_id = s.id
  
left join (
  select  t1.user_id,
          listagg(t2.label, ', ') as pets
  from xxx t1
  joinxxx t2
    on t1.number_value=t2.number_value
    and t1.question_id=t2.question_id
    and t1.question_id = 9
  group by 1) e
  on a.user_id=e.user_id);

select * from {tablename};
""".format(temptable=temptable,
           salt=salt,
           fieldnames=fieldnames,
           tablename=tablename)

In [25]:
print(data_request1)


drop table if exists stg_data_genie.API1_20220214_HZ_TestingforPeter3_93;

create table stg_data_genie.API1_20220214_HZ_TestingforPeter3_93 as (
select  md5(concat(a.user_id, 'tF$SOQjjIoFi')) AS hashed_user_id,
        a.user_id as user_id,
        postal_code, datediff(year, date_of_birth, current_date) as age, ethnicity, gender, firstname, lastname, phone, lifetime_points, created_at, status
from API1_UserID_20220214_HZ_TestingforPeter3_93 a
join prd_edw.vw_user_answers_profiles_by_source b
  on a.user_id = b.user_id
join prd_edw.vw_user_answers_demographics_by_source c
  on a.user_id = c.user_id
left join prd_edw.user_answers_ccpacompliance_by_source d
  on a.user_id=d.user_id
  and d.user_id IS NULL
LEFT JOIN prd_sj.sj_users s
    ON a.user_id = s.id
  
left join (
  select  t1.user_id,
          listagg(t2.label, ', ') as pets
  from prd_sj.sj_answers t1
  join prd_sj.sj_predefined_answers t2
    on t1.number_value=t2.number_value
    and t1.question_id=t2.question_id
    and t1.

In [26]:
data_requests_log_insert="""
INSERT INTO stg_data_genie.data_requests_log
(id, DR_type, request_date, analyst_initials, requester_initials, request_subject, 
 salt, create_timestamp, table_name, query)
VALUES (
  {id_val},
  '{DR_type}',
  '{request_date}',
  '{analyst_initials}',
  '{requester_initials}',
  '{request_subject}',
  '{salt}',
  '{create_timestamp}',
  '{table_name}',
  '{query}');
""".format(id_val=id_val,
           DR_type='API1',
           request_date=date,
           analyst_initials='PG',
           requester_initials=initials,
           request_subject=subject,
           salt=salt,
           create_timestamp=dt.datetime.utcnow(),
           table_name=tablename,
           query=data_request1.replace("'","''"))
pr.exec_commit(data_requests_log_insert) 

In [ ]:
start=dt.datetime.now()

data_request=pr.redshift_to_pandas(data_request1)

end=dt.datetime.now()
print('minutes to complete query:', (end-start).seconds//60)

#### Review & Export

In [ ]:
data_request.head()

In [ ]:
data_request.info()
#sys.getsizeof(data_request)

In [30]:
export_folder="/Users/peter.gurka/Google Drive/My Drive/Self Serve Reporting/API 1 User Demographic Data Append/Outputs/"

In [31]:
csv_export_1 = export_folder + """API1_{0}_{1}_{2}_{3}_CLIENT.csv""".format(initials, subject, date, rand)

In [32]:
data_request.to_csv(csv_export_1)

In [33]:
data_requests_log_update="""
UPDATE stg_data_genie.data_requests_log 
SET complete_timestamp='{complete_timestamp}'
WHERE id={id_val};
""".format(complete_timestamp=dt.datetime.utcnow(),
           id_val=id_val)
pr.exec_commit(data_requests_log_update)

In [34]:
pr.close_up_shop()

In [35]:
job_end=dt.datetime.utcnow()
job_runtime=(job_end-job_start).seconds//60
print('minutes to complete job:', job_runtime)

minutes to complete job: 0


#### Post on Slack

In [36]:
slack_message_end = """{0}'s API1 request for {1} has completed in {2} minutes"""\
                    .format(initials, subject, job_runtime)
slack_data = {'text': slack_message_end}

In [37]:
response = requests.post(
    webhook_url, data = json.dumps(slack_data),
    headers={'Content-Type': 'application/json'})
if response.status_code != 200:
    raise ValueError(
        'Request to slack returned an error %s, the response is:\n%s'
        % (response.status_code, response.text))